In [1]:
import os
import sys
import pandas as pd
import numpy as np
import pycdlib # this library allows to access .iso files (Disc Image Files)

In [2]:
ruta = os.getcwd()

In [3]:
primaria_list = list()

for root, dirs, files in os.walk(ruta):
    for file in files:
        if '.iso' in file:
            # print(os.path.join(root, file))

            iso = pycdlib.PyCdlib()
            iso.open(os.path.join(root, file)) # Renders an absolute path to .iso file
            
            iso.get_file_from_iso('C:\\Users\\Rubén Martínez\\primaria_g14.csv', iso_path='/CSV/RESERVA_PRIMARIA.CSV;1')
            # creates a temporary copy of the content of the "RESERVA_PRIMARIA.CSV" file into the 'primaria_g14.csv' file
            
            primaria_list.append(pd.read_csv('C:\\Users\\Rubén Martínez\\primaria_g14.csv'))
            # appends the content of 'primaria_g14.csv' file into a list before it gets replaced by a new iteration of the loop

            iso.close()
            
primaria = pd.concat(primaria_list)

In [4]:
secundaria_list = list()

for root, dirs, files in os.walk(ruta):
    for file in files:
        if '.iso' in file:
            # print(os.path.join(root, file))

            iso = pycdlib.PyCdlib()
            iso.open(os.path.join(root, file)) # Renders an absolute path to .iso file
            
            iso.get_file_from_iso('C:\\Users\\Rubén Martínez\\secundaria_g14.csv', iso_path='/CSV/RESERVA_SECUNDARIA.CSV;1')
            # creates a temporary copy of the content of the "RESERVA_SECUNDARIA.CSV" file into the 'secundaria_g14.csv' file
            
            secundaria_list.append(pd.read_csv('C:\\Users\\Rubén Martínez\\secundaria_g14.csv'))
            # appends the content of 'secundaria_g14.csv' file into a list before it gets replaced by a new iteration of the loop

            iso.close()
            
secundaria = pd.concat(secundaria_list)

In [5]:
primaria[' FECHA'] = primaria[' FECHA'].str.lstrip()
secundaria[' FECHA'] = secundaria[' FECHA'].str.lstrip()

In [6]:
primaria.dropna(how='all', inplace=True)
secundaria.dropna(how='all', inplace=True)

In [7]:
primaria['DIA'] = primaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[0])
primaria['MES'] = primaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[1])
primaria['ANYO'] = primaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[2])

primaria['FECHA_'] = pd.to_datetime(primaria['ANYO'] + '-' + primaria['MES'] + '-' + primaria['DIA'])
primaria[' HORA'] = primaria[' HORA'].apply(lambda x: int(x))

primaria['FECHA_HORA'] = pd.to_datetime(primaria['FECHA_']) + pd.to_timedelta(primaria[' HORA'], unit='h')

del primaria[' FECHA']
del primaria[' HORA']
del primaria['FECHA_']
del primaria['DIA']
del primaria['MES']
del primaria['ANYO']

In [8]:
secundaria['DIA'] = secundaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[0])
secundaria['MES'] = secundaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[1])
secundaria['ANYO'] = secundaria[' FECHA'].apply(lambda x: x.split()[0].split('/')[2])

secundaria['FECHA_'] = pd.to_datetime(secundaria['ANYO'] + '-' + secundaria['MES'] + '-' + secundaria['DIA'])
secundaria[' HORA'] = secundaria[' HORA'].apply(lambda x: int(x))

secundaria['FECHA_HORA'] = pd.to_datetime(secundaria['FECHA_']) + pd.to_timedelta(secundaria[' HORA'], unit='h')

del secundaria[' FECHA']
del secundaria[' HORA']
del secundaria['FECHA_']
del secundaria['DIA']
del secundaria['MES']
del secundaria['ANYO']

In [9]:
primaria.set_index('FECHA_HORA', inplace=True)
secundaria.set_index('FECHA_HORA', inplace=True)

In [10]:
primaria[' FACTURA_RES_PRI'] = pd.to_numeric(primaria[' FACTURA_RES_PRI'], errors='coerce')
secundaria[' FAC_TOTAL'] = pd.to_numeric(secundaria[' FAC_TOTAL'], errors='coerce')

In [11]:
writer = pd.ExcelWriter('spinning_reserve_g14.xlsx', engine='xlsxwriter')

primaria.resample('D').sum().to_excel(writer, sheet_name='primaria_g14')
secundaria.resample('D').sum().to_excel(writer, sheet_name='secundaria_g14')

writer.save()